# The Wonderful World of Walmart

This notebook begins to explore the retail sales of Walmart over a time period of.  INSERT TIME PERIOD.
We will be predicting sales price per store. 

The data source is below

https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data

First things first, lets import our librarieand take a peek at our data.


In [1]:
import pandas as pd

train = pd.read_csv('Data/train.csv', sep = ',') # read in the training data 
feat = pd.read_csv('Data/features.csv', sep = ',') # Read in feature Data
stores = pd.read_csv('Data/stores.csv', sep = ',') # read in the store data 

#leaving the merge out for a second to look at individual file stats
#Merging it yields 76 million rows



Now that we've got the data inputted, let's begin our EDA process by looking at our datatypes and
overall shape of the data.


In [6]:
#Store Data
print(stores.head().append(stores.tail()), "\n")
print("Count of missing store values:\n", stores.isnull().sum().sort_values(ascending=False), "\n")

#Feature Data
print(feat.head().append(feat.tail()), "\n")
print("Structure of Feature: ",feat.shape,"\n")
print("Summary Statistic:\n",feat.describe(),"\n")
print("Count of missing feature values:\n",feat.isnull().sum().sort_values(ascending=False),"\n")

#Training Data
print(train.head().append(train.tail()),"\n")
print("Structure of train:\n",train.shape,"\n")
print("Summary Statistic:\n",train.describe(),"\n")
print("Count of missing training values:\n",train.isnull().sum().sort_values(ascending=False),"\n")

#df.info()
#df.shape



    Store Type    Size
0       1    A  151315
1       2    A  202307
2       3    B   37392
3       4    A  205863
4       5    B   34875
40     41    A  196321
41     42    C   39690
42     43    C   41062
43     44    C   39910
44     45    B  118221 

Count of missing store values:
 Size     0
Type     0
Store    0
dtype: int64 

      Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0         1  2010-02-05        42.31       2.572        NaN        NaN   
1         1  2010-02-12        38.51       2.548        NaN        NaN   
2         1  2010-02-19        39.93       2.514        NaN        NaN   
3         1  2010-02-26        46.63       2.561        NaN        NaN   
4         1  2010-03-05        46.50       2.625        NaN        NaN   
8185     45  2013-06-28        76.05       3.639    4842.29     975.03   
8186     45  2013-07-05        77.50       3.614    9090.48    2268.58   
8187     45  2013-07-12        79.37       3.614    3789.94    1827.31   

Alright, with some basic statistics above we can now consider merging the datasets into something
more useable.  


In [11]:
#First merge train with stores
df_train = pd.merge(train, stores, on='Store', how='left')
df_train = pd.merge(df_train, feat, on='Store', how='left')
#drop the duplicate holiday column
df_train = df_train.drop(["IsHoliday_y"], axis=1)
processed_train = df_train.fillna(0)
processed_train.loc[processed_train['Weekly_Sales'] < 0.0,'Weekly_Sales'] = 0.0
processed_train.loc[processed_train['MarkDown2'] < 0.0,'MarkDown2'] = 0.0
processed_train.loc[processed_train['MarkDown3'] < 0.0,'MarkDown3'] = 0.0
processed_train.describe()


,Store,Dept,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,7.672574e+07,7.672574e+07,7.672574e+07,7.672574e+07,7.672574e+07,7.672574e+07,7.672574e+07,7.672574e+07,7.672574e+07,7.672574e+07,7.672574e+07,7.672574e+07,7.672574e+07
mean,2.220055e+01,4.426032e+01,1.598147e+04,1.367279e+05,5.882304e+01,3.407146e+00,3.675855e+03,1.286081e+03,8.273969e+02,1.487130e+03,2.140667e+03,1.597750e+02,7.238855e+00
std,1.278528e+01,3.049202e+01,2.271101e+04,6.098051e+04,1.868010e+01,4.311720e-01,7.591808e+03,5.694505e+03,7.807900e+03,4.860260e+03,9.774241e+03,5.843843e+01,2.695273e+00
min,1.000000e+00,1.000000e+00,0.000000e+00,3.487500e+04,-7.290000e+00,2.472000e+00,-2.781450e+03,0.000000e+00,0.000000e+00,0.000000e+00,-1.851700e+02,0.000000e+00,0.000000e+00
25%,1.100000e+01,1.800000e+01,2.079640e+03,9.363800e+04,4.529000e+01,3.039000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.311267e+02,6.338000e+00
50%,2.200000e+01,3.700000e+01,7.612030e+03,1.401670e+05,6.009000e+01,3.521000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.404348e+02,7.567000e+00
75%,3.300000e+01,7.400000e+01,2.020586e+04,2.025050e+05,7.329000e+01,3.743000e+00,5.037950e+03,1.172000e+02,2.891000e+01,9.043400e+02,2.850100e+03,2.122360e+02,8.512000e+00
max,4.500000e+01,9.900000e+01,6.930994e+05,2.196220e+05,1.019500e+02,4.468000e+00,1.031850e+05,1.045195e+05,1.494833e+05,6.747485e+04,7.714481e+05,2.289765e+02,1.431300e+01


In [12]:
print(processed_train.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 76725740 entries, 0 to 76725739
Data columns (total 17 columns):
Store           int64
Dept            int64
Date_x          object
Weekly_Sales    float64
IsHoliday_x     bool
Type            object
Size            int64
Date_y          object
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
dtypes: bool(1), float64(10), int64(3), object(3)
memory usage: 9.8+ GB
None
